<a href="https://colab.research.google.com/github/lionem2018/ML_B4E_homework/blob/main/4%EC%A3%BC%EC%B0%A8_%EA%B3%BC%EC%A0%9C_Saza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 !pip install --upgrade imblearn

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

train_df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/train.csv.zip")
test_df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/test.csv.zip")

In [5]:
def get_preprocessed_data(df):
  df['open_month'] = df['Open Date'].apply(lambda x:int(x.split('/')[0]))
  df['open_day'] = df['Open Date'].apply(lambda x:int(x.split('/')[1]))
  df['open_year'] = df['Open Date'].apply(lambda x:int(x.split('/')[2]))

  return df

train_df = get_preprocessed_data(train_df)
test_df = get_preprocessed_data(test_df)
train_df.head()

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P32,P33,P34,P35,P36,P37,revenue,open_month,open_day,open_year
0,0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,...,4,5,5,4,3,4,5653753.0,7,17,1999
1,1,02/14/2008,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,...,0,0,0,0,0,0,6923131.0,2,14,2008
2,2,03/09/2013,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,...,0,0,0,0,0,0,2055379.0,3,9,2013
3,3,02/02/2012,Tokat,Other,IL,6,4.5,6.0,6.0,4,...,10,6,18,12,12,6,2675511.0,2,2,2012
4,4,05/09/2009,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,...,3,2,3,4,3,3,4316715.0,5,9,2009


In [6]:
train_df = train_df.drop(['Id', 'Open Date'], axis = 1, inplace=False)
test_df = test_df.drop(['Id', 'Open Date'], axis = 1, inplace=False)
train_df.head()

,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,...,P32,P33,P34,P35,P36,P37,revenue,open_month,open_day,open_year
0,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,2,5,...,4,5,5,4,3,4,5653753.0,7,17,1999
1,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,2,5,...,0,0,0,0,0,0,6923131.0,2,14,2008
2,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,3,5,...,0,0,0,0,0,0,2055379.0,3,9,2013
3,Tokat,Other,IL,6,4.5,6.0,6.0,4,4,10,...,10,6,18,12,12,6,2675511.0,2,2,2012
4,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,2,5,...,3,2,3,4,3,3,4316715.0,5,9,2009


In [7]:
train_object_col_names = train_df.dtypes[train_df.dtypes == 'object'].index
test_object_col_names = test_df.dtypes[test_df.dtypes == 'object'].index
object_columns = list(set(train_object_col_names.tolist() + test_object_col_names.tolist()))

encoder = LabelEncoder()
for object_column in object_columns:
  train_df[object_column] = encoder.fit_transform(train_df[object_column])

for object_column in object_columns:
  test_df[object_column] = encoder.fit_transform(test_df[object_column])

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 44 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        137 non-null    int64  
 1   City Group  137 non-null    int64  
 2   Type        137 non-null    int64  
 3   P1          137 non-null    int64  
 4   P2          137 non-null    float64
 5   P3          137 non-null    float64
 6   P4          137 non-null    float64
 7   P5          137 non-null    int64  
 8   P6          137 non-null    int64  
 9   P7          137 non-null    int64  
 10  P8          137 non-null    int64  
 11  P9          137 non-null    int64  
 12  P10         137 non-null    int64  
 13  P11         137 non-null    int64  
 14  P12         137 non-null    int64  
 15  P13         137 non-null    float64
 16  P14         137 non-null    int64  
 17  P15         137 non-null    int64  
 18  P16         137 non-null    int64  
 19  P17         137 non-null    i

In [9]:
from sklearn.model_selection import train_test_split

ftr_train = train_df.drop('revenue', axis=1, inplace=False)
tgt_train = train_df['revenue']

ftr_tr, ftr_val, tgt_tr, tgt_val = train_test_split(ftr_train, tgt_train, test_size=0.2, random_state=0)

In [10]:
# 요구사항 10번

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

rf_reg = RandomForestRegressor(n_estimators=500, random_state=0)
lgbm_reg = LGBMRegressor(n_estimators=500)
regressors = [rf_reg, lgbm_reg]

for regressor in regressors:
  regressor.fit(ftr_tr, tgt_tr)
  preds = regressor.predict(ftr_val)
  rmse_val = mean_squared_error(tgt_val, preds, squared=True)

  print(regressor.__class__.__name__, ' RMSE:', rmse_val)

RandomForestRegressor  RMSE: 9033391751725.385
LGBMRegressor  RMSE: 12101050517411.953


In [11]:
# 요구사항 11번

from sklearn.model_selection import GridSearchCV

rf_reg = RandomForestRegressor(n_estimators=100, random_state=0)
params = {
    'max_depth': [5, 7, 10, 12],
    'max_features': [0.2, 0.4, 0.6, 0.9, 1],
    'min_samples_leaf': [5, 10, 20, 40],
    'min_samples_split': [1, 2, 4, 7]
}
grid_model = GridSearchCV(rf_reg, param_grid = params, scoring='neg_root_mean_squared_error', cv=5, verbose=True)
grid_model.fit(ftr_tr, tgt_tr)
rmse = np.sqrt(-1 * grid_model.best_score_)

Fitting 5 folds for each of 320 candidates, totalling 1600 fits


In [12]:
print('{0} 5 CV 시 최적 평균 RMSE 값: {1}, 최적 alpha:{2}'.format(rf_reg.__class__.__name__, np.round(rmse, 4), grid_model.best_params_))

RandomForestRegressor 5 CV 시 최적 평균 RMSE 값: 1472.2413, 최적 alpha:{'max_depth': 5, 'max_features': 0.6, 'min_samples_leaf': 20, 'min_samples_split': 2}


In [13]:
def get_rmse(y, pred):
  return np.sqrt(mean_squared_error(y, pred))

In [14]:
# 요구사항 12번
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(n_estimators=500, random_state =0, max_depth=5, max_features=0.6, min_samples_leaf=20, min_samples_split=2)
rf_reg.fit(ftr_tr, tgt_tr)
preds = rf_reg.predict(ftr_val)
print("rmse 값:", get_rmse(tgt_val, preds))

rmse 값: 3183968.9938551346
